In [31]:
import time
from influxdb_client_3 import InfluxDBClient3, Point
import certifi
import pandas as pd

## Database Connection

In [32]:
# InfluxDB configuration
token = '8Kph_oFbnNqTvgUfERhTJveJpw1A08-HDsnhftEIYj8HRSUJSmKIdoPZdaIbNeIkG4likhDrOtr5FgyF5W03hg=='
org = "Ethan"
host = "https://us-east-1-1.aws.cloud2.influxdata.com"
client = InfluxDBClient3(host=host, token=token, org=org, ssl_ca_cert=certifi.where())
database = "Test"

## Test Data

In [33]:
# Example sensor data (hardcoded)
dht_sensor_data = [
    {"sensor": "DHT17", "Temp_C": 20.5, "Temp_F": 68.9, "Humidity": 55},
    {"sensor": "DHT18", "Temp_C": 21.0, "Temp_F": 69.8, "Humidity": 60},
    {"sensor": "DHT22", "Temp_C": 19.8, "Temp_F": 67.6, "Humidity": 58},
]

water_sensor_data = [
    {"sensor": "water_sensor", "RTD": 0.98, "pH": 6.5, "EC": 1.1},
    {"sensor": "water_sensor", "RTD": 0.95, "pH": 6.6, "EC": 1.2},
    {"sensor": "water_sensor", "RTD": 0.99, "pH": 6.4, "EC": 1.3},
]

## Data Upload Functions

In [35]:
# Function to upload DHT sensor data to InfluxDB
def upload_dht_data(data):
    for entry in data:
        point = (
            Point("dht_sensors")  # Measurement for DHT sensors
            .tag("sensor", entry["sensor"])  # Tag for sensor type
            .field("Temp_C", entry["Temp_C"])  # Temperature in Celsius
            .field("Temp_F", entry["Temp_F"])  # Temperature in Fahrenheit
            .field("Humidity", entry["Humidity"])  # Humidity level
        )
        client.write(database=database, record=point)
        print(f"Uploaded DHT sensor data from {entry['sensor']}: {entry}")
        time.sleep(1)  # Optional: Delay to simulate real-time data collection

# Function to upload water sensor data to InfluxDB
def upload_water_data(data):
    for entry in data:
        point = (
            Point("water_sensors")  # Measurement for water sensors
            .tag("sensor", entry["sensor"])  # Tag for sensor type
            .field("RTD", entry["RTD"])  # RTD value
            .field("pH", entry["pH"])  # pH value
            .field("EC", entry["EC"])  # Electrical conductivity
        )
        client.write(database=database, record=point)
        print(f"Uploaded water sensor data from {entry['sensor']}: {entry}")
        time.sleep(1)  # Optional: Delay to simulate real-time data collection

## Data Upload

In [36]:
# Upload data for DHT sensors
upload_dht_data(dht_sensor_data)

Uploaded DHT sensor data from DHT17: {'sensor': 'DHT17', 'Temp_C': 20.5, 'Temp_F': 68.9, 'Humidity': 55}
Uploaded DHT sensor data from DHT18: {'sensor': 'DHT18', 'Temp_C': 21.0, 'Temp_F': 69.8, 'Humidity': 60}
Uploaded DHT sensor data from DHT22: {'sensor': 'DHT22', 'Temp_C': 19.8, 'Temp_F': 67.6, 'Humidity': 58}


In [37]:
# Upload data for water sensors
upload_water_data(water_sensor_data)

Uploaded water sensor data from water_sensor: {'sensor': 'water_sensor', 'RTD': 0.98, 'pH': 6.5, 'EC': 1.1}
Uploaded water sensor data from water_sensor: {'sensor': 'water_sensor', 'RTD': 0.95, 'pH': 6.6, 'EC': 1.2}
Uploaded water sensor data from water_sensor: {'sensor': 'water_sensor', 'RTD': 0.99, 'pH': 6.4, 'EC': 1.3}


## Sample Queries

In [27]:
# Query
query = """
SELECT *
FROM 'dht_sensors'
WHERE "sensor" = 'DHT17'
"""

# Execute the query
table = client.query(query=query, database="Test", language='sql')

# Format results
df = table.to_pandas().sort_values(by="time")
# 'time' timezone conversion
df['time'] = pd.to_datetime(df['time']).dt.tz_localize('UTC')
df['time'] = df['time'].dt.tz_convert('US/Eastern')

df

,Humidity,Temp_C,Temp_F,sensor,time
0,55,20.5,68.9,DHT17,2024-09-05 17:16:44.881806030-04:00


In [38]:
# Query
query = """
SELECT *
FROM 'dht_sensors'
"""

# Execute the query
table = client.query(query=query, database="Test", language='sql')

# Format results
df = table.to_pandas().sort_values(by="time")
# 'time' timezone conversion
df['time'] = pd.to_datetime(df['time']).dt.tz_localize('UTC')
df['time'] = df['time'].dt.tz_convert('US/Eastern')

df

,Humidity,Temp_C,Temp_F,sensor,time
0,55,20.5,68.9,DHT17,2024-09-06 09:58:44.943561814-04:00
1,60,21.0,69.8,DHT18,2024-09-06 09:58:46.062427949-04:00
2,58,19.8,67.6,DHT22,2024-09-06 09:58:47.118459821-04:00


In [40]:
# Query
query = """
SELECT *
FROM 'water_sensors'
"""

# Execute the query
table = client.query(query=query, database="Test", language='sql')

# Format results
df = table.to_pandas().sort_values(by="time")
# 'time' timezone conversion
df['time'] = pd.to_datetime(df['time']).dt.tz_localize('UTC')
df['time'] = df['time'].dt.tz_convert('US/Eastern')

df

,EC,RTD,pH,sensor,time
0,1.1,0.98,6.5,water_sensor,2024-09-06 09:58:54.373186114-04:00
1,1.2,0.95,6.6,water_sensor,2024-09-06 09:58:55.491489392-04:00
2,1.3,0.99,6.4,water_sensor,2024-09-06 09:58:56.541043596-04:00
